# Rekurrens neurális hálózatok

Az előző modulban gazdag szemantikai reprezentációkat használtunk a szövegekhez, és egy egyszerű lineáris osztályozót helyeztünk az embeddingek fölé. Ez az architektúra a mondatokban szereplő szavak összesített jelentését ragadja meg, de nem veszi figyelembe a szavak **sorrendjét**, mivel az embeddingek fölötti aggregációs művelet eltávolította ezt az információt az eredeti szövegből. Mivel ezek a modellek nem képesek a szavak sorrendjét modellezni, nem tudnak megoldani összetettebb vagy kétértelmű feladatokat, például szövegalkotást vagy kérdés-megválaszolást.

Ahhoz, hogy a szövegszekvencia jelentését megragadjuk, egy másik neurális hálózati architektúrát kell használnunk, amelyet **rekurrens neurális hálózatnak** (RNN) nevezünk. Az RNN-ben mondatunkat egy szimbólumonként haladva adjuk át a hálózaton, és a hálózat egy **állapotot** hoz létre, amelyet aztán a következő szimbólummal együtt ismét átadunk a hálózatnak.

Adott $X_0,\dots,X_n$ tokenekből álló bemeneti szekvencia esetén az RNN egy neurális hálózati blokkokból álló szekvenciát hoz létre, és ezt a szekvenciát végponttól végpontig tanítja visszaterjesztés segítségével. Minden hálózati blokk egy $(X_i,S_i)$ párt kap bemenetként, és $S_{i+1}$-et állít elő eredményként. A végső állapot $S_n$ vagy a kimenet $X_n$ egy lineáris osztályozóba kerül, amely előállítja az eredményt. Az összes hálózati blokk ugyanazokat a súlyokat osztja meg, és egyetlen visszaterjesztési lépés során végponttól végpontig tanítják őket.

Mivel az állapotvektorok $S_0,\dots,S_n$ átmennek a hálózaton, képes megtanulni a szavak közötti szekvenciális függőségeket. Például, ha a *nem* szó valahol megjelenik a szekvenciában, megtanulhatja bizonyos elemek tagadását az állapotvektoron belül, ami tagadást eredményez.

> Mivel az RNN blokkok súlyai a képen megosztottak, ugyanaz a kép egyetlen blokként is ábrázolható (a jobb oldalon) egy rekurrens visszacsatolási hurokkal, amely a hálózat kimeneti állapotát visszavezeti a bemenethez.

Nézzük meg, hogyan segíthetnek a rekurrens neurális hálózatok a hírek adatállományának osztályozásában.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Egyszerű RNN osztályozó

Egyszerű RNN esetén minden rekurrens egység egy egyszerű lineáris hálózat, amely a bemeneti vektor és az állapotvektor összefűzött változatát veszi, és egy új állapotvektort hoz létre. A PyTorch ezt az egységet az `RNNCell` osztállyal képviseli, míg az ilyen cellák hálózatát az `RNN` rétegként.

Egy RNN osztályozó definiálásához először egy beágyazási réteget alkalmazunk, hogy csökkentsük a bemeneti szókincs dimenzióját, majd erre építjük az RNN réteget:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Megjegyzés:** Itt egyszerűség kedvéért nem használtunk előre betanított beágyazási réteget, de még jobb eredmények érdekében használhatunk előre betanított beágyazási réteget Word2Vec vagy GloVe beágyazásokkal, ahogy az előző egységben leírtuk. A jobb megértés érdekében érdemes lehet ezt a kódot úgy módosítani, hogy előre betanított beágyazásokat használjon.

Esetünkben párnázott adatbetöltőt fogunk használni, így minden batch azonos hosszúságú párnázott szekvenciákat fog tartalmazni. Az RNN réteg beágyazási tenzorok szekvenciáját veszi, és két kimenetet állít elő:
* $x$ az RNN cellák kimeneteinek szekvenciája minden lépésnél
* $h$ a szekvencia utolsó elemének végső rejtett állapota

Ezután egy teljesen összekapcsolt lineáris osztályozót alkalmazunk, hogy megkapjuk az osztályok számát.

> **Megjegyzés:** Az RNN-eket elég nehéz betanítani, mivel amikor az RNN cellák a szekvencia hosszán keresztül kibonthatók, a visszaterjesztésben részt vevő rétegek száma meglehetősen nagy lesz. Ezért kis tanulási rátát kell választanunk, és nagyobb adathalmazon kell betanítanunk a hálózatot, hogy jó eredményeket érjünk el. Ez elég hosszú időt vehet igénybe, ezért ajánlott GPU-t használni.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Hosszú és rövid távú memória (LSTM)

A klasszikus RNN-ek egyik fő problémája az úgynevezett **eltűnő gradiens** probléma. Mivel az RNN-eket egyetlen visszaterjesztési lépésben, végponttól végpontig tanítják, nehézséget okoz a hiba propagálása a hálózat első rétegeihez, így a hálózat nem tudja megtanulni a távoli tokenek közötti kapcsolatokat. Ennek a problémának az elkerülésére az egyik megoldás az **explicit állapotkezelés** bevezetése úgynevezett **kapuk** használatával. Két legismertebb ilyen architektúra: a **Hosszú és rövid távú memória** (LSTM) és a **Kapuzott reléegység** (GRU).

![Kép, amely egy hosszú és rövid távú memória cella példáját mutatja](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Az LSTM hálózat felépítése hasonló az RNN-hez, de két állapotot továbbítanak rétegről rétegre: az aktuális állapotot $c$, és a rejtett vektort $h$. Minden egységnél a rejtett vektor $h_i$ összefűződik a bemenettel $x_i$, és ezek irányítják, hogy mi történik az állapottal $c$ a **kapuk** segítségével. Minden kapu egy neurális hálózat szigmoid aktivációval (kimenet tartománya $[0,1]$), amely bitmaszkként értelmezhető, amikor megszorozzuk az állapotvektorral. Az alábbi kapuk léteznek (a fenti képen balról jobbra):
* **felejtő kapu** veszi a rejtett vektort, és meghatározza, hogy az $c$ vektor mely komponenseit kell elfelejteni, és melyeket kell továbbadni.
* **bemeneti kapu** vesz némi információt a bemenetből és a rejtett vektorból, majd beilleszti az állapotba.
* **kimeneti kapu** átalakítja az állapotot egy lineáris rétegen keresztül $\tanh$ aktivációval, majd kiválasztja annak néhány komponensét a rejtett vektor $h_i$ segítségével, hogy új állapotot hozzon létre $c_{i+1}$.

Az állapot $c$ komponensei úgy tekinthetők, mint bizonyos jelzők, amelyeket be- és kikapcsolhatunk. Például, amikor egy *Alice* nevű szót találunk a sorozatban, feltételezhetjük, hogy női karakterre utal, és bekapcsolhatjuk az állapotban azt a jelzőt, hogy nőnemű főnév van a mondatban. Amikor később találkozunk az *és Tom* kifejezéssel, bekapcsolhatjuk azt a jelzőt, hogy többes számú főnév van jelen. Így az állapot manipulálásával nyomon követhetjük a mondatrészek nyelvtani tulajdonságait.

> **Note**: Egy kiváló forrás az LSTM belső működésének megértéséhez Christopher Olah cikke, [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Bár az LSTM cella belső szerkezete bonyolultnak tűnhet, a PyTorch elrejti ezt a megvalósítást az `LSTMCell` osztályban, és az `LSTM` objektumot biztosítja az egész LSTM réteg reprezentálására. Így az LSTM osztályozó megvalósítása nagyon hasonló lesz az egyszerű RNN-hez, amelyet korábban láttunk:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Csomagolt szekvenciák

Példánkban minden szekvenciát nullvektorokkal kellett kitöltenünk a minibatch-ben. Ez némi memóriapazarlást eredményez, de az RNN-ek esetében még kritikusabb, hogy a kitöltött bemeneti elemekhez további RNN cellák jönnek létre, amelyek részt vesznek a tanításban, de nem hordoznak semmilyen fontos bemeneti információt. Sokkal jobb lenne, ha az RNN-t csak a tényleges szekvenciahosszra képeznénk.

Ehhez a PyTorch-ban bevezettek egy speciális formátumot a kitöltött szekvenciák tárolására. Tegyük fel, hogy van egy kitöltött minibatch bemenetünk, amely így néz ki:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Itt a 0 a kitöltött értékeket jelöli, és a bemeneti szekvenciák tényleges hossza `[5,3,1]`.

Ahhoz, hogy hatékonyan képezzük az RNN-t kitöltött szekvenciával, azt szeretnénk, hogy az első RNN cellacsoportot nagy minibatch-csel (`[1,6,9]`) kezdjük el tanítani, majd befejezzük a harmadik szekvencia feldolgozását, és rövidebb minibatch-ekkel (`[2,7]`, `[3,8]`) folytassuk a tanítást, és így tovább. Így a csomagolt szekvencia egyetlen vektorként van ábrázolva – esetünkben `[1,6,9,2,7,3,8,4,5]`, valamint egy hosszvektorral (`[5,3,1]`), amelyből könnyen visszaállíthatjuk az eredeti kitöltött minibatch-et.

A csomagolt szekvencia előállításához használhatjuk a `torch.nn.utils.rnn.pack_padded_sequence` függvényt. Minden rekurrens réteg, beleértve az RNN-t, LSTM-et és GRU-t, támogatja a csomagolt szekvenciákat bemenetként, és csomagolt kimenetet állít elő, amelyet a `torch.nn.utils.rnn.pad_packed_sequence` segítségével dekódolhatunk.

Ahhoz, hogy csomagolt szekvenciát tudjunk előállítani, át kell adnunk a hosszvektort a hálózatnak, ezért egy másik függvényre van szükségünk a minibatch-ek előkészítéséhez:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

A tényleges hálózat nagyon hasonló lenne a fentebb említett `LSTMClassifier`-hez, de a `forward` metódus mind a kitöltött minibatch-et, mind a szekvenciahosszak vektorát megkapja bemenetként. Az embedding kiszámítása után létrehozzuk a csomagolt szekvenciát, továbbítjuk az LSTM rétegnek, majd az eredményt visszacsomagoljuk.

> **Megjegyzés**: Valójában nem használjuk a visszacsomagolt `x` eredményt, mivel a következő számításokban a rejtett rétegek kimenetét használjuk. Ezért ezt a visszacsomagolást teljesen el is hagyhatjuk ebből a kódból. Azért helyeztük ide, hogy könnyen módosíthasd a kódot, ha a hálózat kimenetére lenne szükséged további számítások során.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Megjegyzés:** Észrevehetted a `use_pack_sequence` paramétert, amelyet az oktatási függvénynek adunk át. Jelenleg a `pack_padded_sequence` függvény megköveteli, hogy a hosszúsági szekvencia tensor a CPU eszközön legyen, ezért az oktatási függvénynek el kell kerülnie, hogy a hosszúsági szekvencia adatokat a GPU-ra mozgassa az oktatás során. Megnézheted a `train_emb` függvény implementációját a [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) fájlban.


## Kétirányú és többrétegű RNN-ek

Példáinkban minden rekurrens hálózat egy irányban működött, a szekvencia elejétől a végéig. Ez természetesnek tűnik, mivel hasonlít arra, ahogyan olvasunk vagy beszédet hallgatunk. Azonban sok gyakorlati esetben véletlenszerűen hozzáférhetünk a bemeneti szekvenciához, így érdemes lehet a rekurrens számítást mindkét irányban futtatni. Az ilyen hálózatokat **kétirányú** RNN-eknek nevezzük, és létrehozhatók az `bidirectional=True` paraméter átadásával az RNN/LSTM/GRU konstruktorának.

Kétirányú hálózat esetén két rejtett állapotvektorra van szükségünk, egy-egy az egyes irányokhoz. A PyTorch ezeket a vektorokat egy kétszer nagyobb méretű vektorként kódolja, ami elég kényelmes, mivel általában a kapott rejtett állapotot egy teljesen összekapcsolt lineáris réteghez továbbítjuk, és csak figyelembe kell venni ezt a méretnövekedést a réteg létrehozásakor.

A rekurrens hálózat, legyen az egyirányú vagy kétirányú, bizonyos mintázatokat rögzít egy szekvencián belül, és ezeket tárolhatja az állapotvektorban vagy továbbíthatja a kimenetbe. Akárcsak a konvolúciós hálózatok esetében, egy másik rekurrens réteget építhetünk az első fölé, hogy magasabb szintű mintázatokat rögzítsünk, amelyeket az első réteg által kinyert alacsony szintű mintázatokból építünk fel. Ez vezet el minket a **többrétegű RNN** fogalmához, amely két vagy több rekurrens hálózatból áll, ahol az előző réteg kimenete bemenetként kerül a következő réteghez.

![Kép egy többrétegű hosszú-rövid távú memória RNN-ről](../../../../../translated_images/hu/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Kép Fernando López [ezen csodálatos bejegyzéséből](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3)*

A PyTorch megkönnyíti az ilyen hálózatok létrehozását, mivel csak az `num_layers` paramétert kell átadni az RNN/LSTM/GRU konstruktorának, hogy automatikusan több rekurrens réteget építsen. Ez azt is jelenti, hogy a rejtett/állapotvektor mérete arányosan növekedni fog, és ezt figyelembe kell venni a rekurrens rétegek kimenetének kezelésénél.


## RNN-ek más feladatokra

Ebben az egységben láttuk, hogy az RNN-ek használhatók szekvenciaosztályozásra, de valójában sok más feladatot is képesek kezelni, például szöveggenerálást, gépi fordítást és még sok mást. Ezeket a feladatokat a következő egységben fogjuk megvizsgálni.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
